<a href="https://colab.research.google.com/github/mahtabmirhaj/job_vacancies/blob/main/job_vacancies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Libraries
%%capture 
import requests
import json
import pandas as pd
from datetime import date, timedelta
import datetime
import calendar
import time
import urllib3
urllib3.disable_warnings()
import warnings
warnings.filterwarnings('ignore')
import inspect
import base64
pd.options.mode.chained_assignment = None
!pip install hurry.filesize # For showing the size in Kb, Mb ,...
from hurry.filesize import size, si 
import random
import csv
import glob
import os
import re
from concurrent.futures import ThreadPoolExecutor


In [ ]:
# Global Variables 
# To keep track of all time and traffic usage
total_use = 0
time_consumed = 0

location = 'Bonn'

# Read parameters from the list
file1 = open('/content/Parameters.txt', 'r')
Lines = file1.readlines()
param = {}
for line in Lines:
  if '#' not in line and '*' not in line:
    a = re.split(r'-|:| |\n|\r', line)
    a = [x for x in a if x != '']
    param[a[0]] = a[1]

# Creating file with today's date as name 
today = date.today()
directory_name = 'Files_'+ str(today)
# Creating a directory to save all csv files into it
!rm -rf $directory_name # Delete (old) Files directory if there is one
!mkdir $directory_name # Create a new one (we need $ to pass a variable for the name of directory)

In [ ]:
# Get Token for Authentcation
def get_jwt():
    headers = {
        'User-Agent': 'Jobsuche/2.9.2 (de.arbeitsagentur.jobboerse; build:1077; iOS 15.1.0) Alamofire/5.4.4',
        'Host': 'rest.arbeitsagentur.de',
        'Connection': 'keep-alive',
        'Content-Type': 'application/x-www-form-urlencoded; charset=utf-8',
    }

    data = {
      'client_id': 'c003a37f-024f-462a-b36d-b001be4cd24a',
      'client_secret': '32a39620-32b3-4307-9aa1-511e3d7f48a8',
      'grant_type': 'client_credentials'
    }

    response = requests.post('https://rest.arbeitsagentur.de/oauth/gettoken_cc', headers=headers, data=data, verify=False)

    return response.json()

In [ ]:
# Search for jobs, different parameters can be find in https://jobsuche.api.bund.dev/
def search(token, location, param):
    global total_use

    params = [(i, param[i]) for i in param.keys()]
    params.append(('wo', location))
    params = tuple(params)


    headers = {
        'User-Agent': 'Jobsuche/2.9.2 (de.arbeitsagentur.jobboerse; build:1077; iOS 15.1.0) Alamofire/5.4.4',
        'Host': 'rest.arbeitsagentur.de',
        'OAuthAccessToken': token,
        'Connection': 'keep-alive',
    }

    response = requests.get('https://rest.arbeitsagentur.de/jobboerse/jobsuche-service/pc/v4/app/jobs', headers=headers, params=params, verify=False)
    # Response's time
    print(f"Job_Search_Response_time: {response.elapsed.total_seconds()}s")
    # Response's body's size
    print(f'Job_Search_Response_size: {size(len(response.content), system=si)} \n')
    total_use += len(response.content)
    
    return response

In [ ]:
# Get the full job_details of each hashId
def job_details(hashId):
    global total_use

    token = get_jwt()
    
    headers = {
        'User-Agent': 'Jobsuche/2.9.3 (de.arbeitsagentur.jobboerse; build:1078; iOS 15.1.0) Alamofire/5.4.4',
        'Host': 'rest.arbeitsagentur.de',
        'OAuthAccessToken': token["access_token"],
        'Connection': 'keep-alive',
    }

    response = requests.get(
        f'https://rest.arbeitsagentur.de/jobboerse/jobsuche-service/pc/v2/jobdetails/{(base64.b64encode(hashId.encode())).decode("UTF-8")}',
        headers=headers, verify=False)
    
    total_use += len(response.content)
    return response

In [ ]:
# Separating different attributes from json in csv
def separate_multi_val(df):
  multi_val_attribute = ['arbeitsorte', 'ausbildungen', 'fertigkeiten']
  df_temp = df.copy()
  j = 0
  for i in range(len(df_temp.columns)):
    if df_temp.columns[i] in multi_val_attribute:
      for st in range(len(df_temp[df_temp.columns[i]].apply(pd.Series).columns)):
        
        test_df = pd.json_normalize(df[df_temp.columns[i]].str[st])

        j = df.columns.get_loc(df_temp.columns[i])
        for k in range(len(test_df.columns)):
          new_name = df_temp.columns[i]+'_'+str(st+1)+'.'+ test_df.columns[k]
          df.insert(j, new_name, test_df[test_df.columns[k]])
          j += 1

      df = df.drop(columns=df_temp.columns[i])

  return df

In [ ]:
# From today to a week ago, find the number of new ads on each day to find a relation between weekdays and number of new ads
def weekly_pattern(response):
  global location
  print(f'Today: {str(date.today())}, Location: {location}')
  j = 0
  for i in range(7):
    day_of_week = calendar.day_name[int(date.today().weekday()) - i]
    print(f"New ads published on {day_of_week} :", int(response.json()['facetten']['veroeffentlichtseit']['counts'][str(i)]) - j)
    j = int(response.json()['facetten']['veroeffentlichtseit']['counts'][str(i)])


In [ ]:
# Using hashIds from the job that were posted before, check if those ads are still on the API and the positions are 
# still online.
def check_old_ads(Old_Open, Old_Closed, token):
  tday = datetime.date.today()
  close_date = str(tday)[:-3]
  closed_hashId, open_date, status, closed_date = [], [], [], []

  # Use multiple Threads
  with ThreadPoolExecutor(max_workers=100) as pool:
    response_list = list(pool.map(job_details,Old_Open['hashId'].tolist()))
  
  for i in range(len(response_list)):
    if response_list[i].status_code == 404:
      Old_Open.iloc[i]['Status'] = 'Closed'
      closed_hashId.append(Old_Open.iloc[i]['hashId'])
      open_date.append(Old_Open.iloc[i]['Published'])
      status.append('Closed')
      closed_date.append(str(date.today()))


  # Update Closed.csv and Open.csv
  df_closed_today = {'hashId': closed_hashId, 'Published': open_date, 'Status': status, 'End date': closed_date}
  New_Closed = pd.concat([pd.DataFrame(df_closed_today), Old_Closed], ignore_index=True)
  New_Closed.reset_index(drop = True, inplace = True)
  Still_Open = Old_Open[Old_Open['Status'] != 'Closed']
  Still_Open.reset_index(drop = True, inplace = True)
  
  return Still_Open, New_Closed

In [ ]:
# Updating close jobs statuses and End dates in PLZ.csv that they belong
def Update_Jobs_PLZs_Closed(df_closed_today):
  df_Closed = df_closed_today.copy()
  hashIdlist = df_Closed['hashId'].tolist()
  endDate = df_Closed['End date'].tolist()
  published =  df_Closed["Published"].tolist() 
  # Get unique dates in df_closed
  unique_namedate = df_Closed["Published"].unique().tolist()
  compelete_unique_namedate = []
  # change date format from d-m-y to Y-m-d to match the names of CSV files
  for k in unique_namedate:
    name = datetime.datetime.strptime(k, "%d-%m-%y").strftime("%Y-%m-%d")
    compelete_unique_namedate.append('/content/'+name+'.csv')
    
  # Update
  for j, k in enumerate(compelete_unique_namedate):
    df = pd.read_csv(k)
    for i, b in enumerate(published):
        hash_id = hashIdlist[i]
        if hash_id in df['hashId'].values and b == unique_namedate[j]:
          indexnumber= df.index[df['hashId']==hash_id]
          df["End date"][indexnumber] = endDate[i]
          df["status"][indexnumber] = "Closed"
    df.to_csv(k, index = False) 


❗ Write all functions above this text 👆


---

❗ Write main codes and function calls under this text 👇

In [ ]:
# Main ----------------------------------------------- 

# get a token
token = get_jwt()

# Job_search
response = search(token["access_token"], 'Bonn', param)
result = json.dumps(response.json(), sort_keys=True, indent=4)
print(f'How many job offers:', len(response.json()['stellenangebote']), '\n')
print(result)

Job_Search_Response_time: 1.092794s
Job_Search_Response_size: 95K 

How many job offers: 100 

{
    "facetten": {
        "angebotsart": {
            "counts": {
                "1": 221,
                "2": 3,
                "4": 12
            },
            "maxCount": 236
        },
        "arbeitgeber": {
            "counts": {
                "ABSOLUT PersonalManagement GmbH": 6,
                "AFC Consulting Group AG": 1,
                "Abels Fr\u00fcchte Welt GmbH": 1,
                "Adecco Personaldienstleistungen GmbH": 8,
                "Allgeier Experts GmbH": 1,
                "Allgeier IT GmbH": 2,
                "Autohaus Dresen GmbH": 1,
                "BDO AG Wirtschaftspr\u00fcfungsgesellschaft": 2,
                "BUHL Personal GmbH - Niederlassung Bonn": 11,
                "BUHL Services GmbH - Abt. Online Marketing - Bonn": 1,
                "Beyss Architekten GmbH": 2,
                "BleckmannSchulze GmbH Care": 4,
                "Bonn Marrio

In [ ]:
start = time.time() # Keep track of program running time as well 

# Find weekly pattern in number of the job ads
weekly_pattern(response)

end = time.time()
time_consumed += end-start;

Today: 2022-05-23, Location: Bonn
New ads published on Monday : 221
New ads published on Sunday : 42
New ads published on Saturday : 70
New ads published on Friday : 345
New ads published on Thursday : 258
New ads published on Wednesday : 192
New ads published on Tuesday : 229


In [ ]:
# Used fixed Plz that are in Used_Plz.csv
Loc_Plz = pd.read_csv('/content/Used_Plz.csv')
Loc_Plz_List = list(Loc_Plz['0'])
print(Loc_Plz_List)

[88167, 39446, 14778, 97762, 91746, 29413, 29451, 67759, 31542, 58840, 48720, 16341, 56291, 7619, 2699, 73650, 7336, 1097, 55270, 27446, 72411, 7774, 54597, 84089, 6295, 91244, 96515, 83567, 26345, 79252, 39615, 57539, 39130, 7570, 1156, 31542, 95183, 4936, 58706, 1994, 7646, 54673, 45128, 4928, 71394, 13355, 87645, 77791, 6347, 78733, 6408, 31595, 57627, 99330, 25582, 98617, 47228, 24811, 91785, 31629, 18311, 86150, 49457, 74076, 27412, 54472, 87640, 48431, 23863, 4626, 6667, 1990, 7349, 45886, 17391, 23883, 8523, 27333, 63599, 65558, 23823, 4838, 31655, 26409, 68549, 27412, 54689, 83024, 54298, 18211, 38159, 39398, 25924, 15518, 39240, 14612, 15831, 97522, 94155, 54608, 54441, 12689, 50933, 7907, 45892, 91183, 56294, 18299, 4178, 6528, 76872, 20255, 21465, 21483, 82544, 24634, 16567, 15366, 57639, 3055, 87654, 95689, 15518, 66131, 88709, 7646, 83365, 26605, 24232, 15926, 54673, 31547, 24999, 41236, 12349, 93470, 55595, 25842, 82432, 71566, 17268, 73278, 76275, 34454, 23919, 19209, 18

In [ ]:
start = time.time() # Keep track of program running time as well 

# Searching jobs based on postal codes; showing how many jobs are in each postal code
# Getting hashIdlist based on postalcodes
hashIdList = {}
parameters = []
token = get_jwt()
for i in Loc_Plz_List:
  k = []
  j, counter = 0, 0
  ebene = "plz_ebene_" + str(len(str(i)))
  # Search based on postal codes
  searchjobs = search(token["access_token"], i, param)
  if searchjobs.ok:
    if 'stellenangebote' in searchjobs.json():
      print(f'Postal code {i} : {searchjobs.json()["facetten"][ebene]["counts"][str(i)]} jobs')
      hashIdList[i] = []
      k = searchjobs.json()["facetten"][ebene]["counts"][str(i)]
      # Get the jobs that has plz same as i
      while j < k and counter<100 :
        if 'plz' in searchjobs.json()["stellenangebote"][j]['arbeitsort']:
          if searchjobs.json()["stellenangebote"][j]['arbeitsort']['plz'] == str(i):
            hashId = searchjobs.json()["stellenangebote"][j]["hashId"]
            hashIdList[i].append(hashId)
            j = j + 1
        counter = counter + 1
      # How many hashIds it actually finds, is it equal to k
      # print(len(hashIdList[i]))
    else:
      print(f'No job_search return with plz = {i} \n')
    print('-------- \n')
    
       
end = time.time()
time_consumed += end-start;      

Streaming output truncated to the last 5000 lines.

-------- 

Job_Search_Response_time: 1.057085s
Job_Search_Response_size: 5K 

Postal code 44629 : 2 jobs
-------- 

Job_Search_Response_time: 1.038014s
Job_Search_Response_size: 2K 

No job_search return with plz = 56379 

-------- 

Job_Search_Response_time: 1.195287s
Job_Search_Response_size: 15K 

Postal code 29221 : 11 jobs
-------- 

Job_Search_Response_time: 1.045987s
Job_Search_Response_size: 107B 

No job_search return with plz = 8301 

-------- 

Job_Search_Response_time: 1.051458s
Job_Search_Response_size: 10K 

Postal code 26316 : 6 jobs
-------- 

Job_Search_Response_time: 1.09994s
Job_Search_Response_size: 21K 

Postal code 85521 : 20 jobs
-------- 

Job_Search_Response_time: 1.180193s
Job_Search_Response_size: 34K 

Postal code 80336 : 37 jobs
-------- 

Job_Search_Response_time: 1.085497s
Job_Search_Response_size: 8K 

Postal code 91541 : 5 jobs
-------- 

Job_Search_Response_time: 1.037843s
Job_Search_Response_size: 2K

In [ ]:
start = time.time() 
# Keep track of program running time as well 
# Job_details 
# call Job_details request for each job's hashId based oo postal codes and save them in a list with thread
job_detailslist, job_ids, checked_csv = [], [], []
for i in hashIdList.keys():
  if len(hashIdList[i]) != 0:
      with ThreadPoolExecutor(max_workers=100) as pool:
        response_list = list(pool.map(job_details,hashIdList[i]))
      for k, job_Details in enumerate(response_list):
        if job_Details.ok:
          hashId = hashIdList[i][k]
          # Add new row to checked_csv
          checked_csv.append({ 'hashId': hashId, 'Published': str(date.today()), 
                        'Status': 'Open'}) 
          csv_name = str(i) + '.csv'
          job_detailslist.append(job_Details.json())
          job_ids.append(csv_name)
end = time.time()
time_consumed += end-start          

In [ ]:
start = time.time() # Keep track of program running time as well 

# Checking available jobs from Open_(date).csv and add closed job to Closed_(date).csv
# If the month has changed, create new CSVs
path = r'/content/'
Files_months = glob.glob(path + "/*.csv")
# This month day
dt = str(today)[:-3]
# Last month date
last_day_of_prev_month = date.today().replace(day=1) - timedelta(days=1)
start_day_of_prev_month = date.today().replace(day=1) - timedelta(days=last_day_of_prev_month.day)
old_date = str(start_day_of_prev_month)[:-3]
flag = False
token = get_jwt()
# Month did not change, use current Open and Closed CSV files for this month
for filename in Files_months: 
  if filename == '/content/Open_' + dt + '.csv' :
    flag = True
    Old_Open = pd.read_csv(filename)
    Old_Closed = pd.read_csv('/content/Closed_' + dt + '.csv')
    Still_Open, Closed = check_old_ads(Old_Open, Old_Closed, token)
    Closed.to_csv(directory_name + '/Closed_' + dt +'.csv', index = False)
    break
# New month has been started, created new Open and Closed CSV files
if not flag:
  Old_Open = pd.read_csv('/content/Open_' + old_date + '.csv')
  Old_Closed = pd.DataFrame()
  Still_Open, Closed = check_old_ads(Old_Open, Old_Closed, token)
  Closed.to_csv(directory_name + '/Closed_' + dt +'.csv', index = False)
  


end = time.time()
time_consumed += end-start

In [ ]:
# Creating csv files

# Creating a CSV having all the job details from today
df = pd.json_normalize(job_detailslist)
df = separate_multi_val(df)     
df.insert(0, "Id", job_ids) 
df['status'] = ['Open' for i in range(len(df))]
df['End date'] = [' ' for i in range(len(df))]
# Replace the new line characters, so the csv file does not break rows
df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)
csv_name = directory_name + '/' + str(date.today()) + '.csv'
df.to_csv(csv_name, index=False)

# Update Open.csv
# Creating a csv file having all Open hashIds, dates, and statuses to check them next week 
df3 = pd.DataFrame.from_records(checked_csv)
# Add checked_csv to Open.csv
tday = datetime.date.today()
daytoday = str(tday)[:-3]
New_Open = pd.concat([df3, Still_Open], ignore_index=True) 
New_Open.to_csv(directory_name + '/Open_'+ daytoday + '.csv' , index=False)     


In [ ]:
%%capture 
# %%capture is used for colab not showing the output cell of a block

# Create a zip file from Files directory to download it
zip_file = directory_name +'.zip'
!rm -rf $zip_file # Delete (old) Files.zip first if there is one
!zip -r /content/$zip_file /content/$directory_name

In [ ]:
# Total time and traffic usage of all the responses
print(f'Total_Response_size: {size(total_use, system=si)} \n')
print(f'Total_runtime: {round(time_consumed,2)}s \n')

Total_Response_size: 113M 

Total_runtime: 3614.8s 

